In [3]:
using DataFrames, DataFramesMeta, Optim, Revise, Serialization, DebuggingUtilities, LinearAlgebra
using JuMP, Ipopt

In [1]:
string(@__DIR__) in LOAD_PATH || push!(LOAD_PATH, @__DIR__)
using BLPmodule; const m = BLPmodule;

In [4]:
test1state = false
if test1state 
    dirpath = "/export/storage_adgandhi/MiscLi/factract/analysis"
else
    dirpath = "jls"
end;
ec = deserialize("$dirpath/ec.jls");

---
### jump

In [5]:
t_indexer = [[] for _ in 1:length(ec.firms)]
for tt in eachindex(ec.tracts)
    for ind in ec.tracts[tt].inds
        push!(t_indexer[ind], tt)
    end
end

In [8]:
n_firms = length(ec.firms);
n_firms_intract = [t.n_firms for t in ec.tracts]
n_tracts = length(ec.tracts);
nX = 2
nD = 2
X = [t.X[:,1:nX] for t in ec.tracts]
D = [t.D[:,1:nD] for t in ec.tracts]
j_indexer = [t.inds for t in ec.tracts]
M = [t.M for t in ec.tracts]
q_obs = ec.q_obs;

In [9]:
model = Model(Ipopt.Optimizer)
@variables(model, begin
    θx[1:nX]
    θd[1:nD]
end)

u = @NLexpression(model,
    [tt in 1:n_tracts, jj in 1:n_firms_intract[tt]],
    sum(θx[ix] * X[tt][jj,ix] for ix in 1:nX) + sum(θd[id] * D[tt][jj,id] for id in 1:nD))


expu = @NLexpression(model,
    [tt in 1:n_tracts, jj in 1:n_firms_intract[tt]],
    exp(u[tt,jj]))

mktdenom = @NLexpression(model,
    [tt in 1:n_tracts],
    1+ sum(expu[tt,jj] for jj in 1:n_firms_intract[tt]))


mktq = @NLexpression(model,
    [tt in 1:n_tracts, jj in 1:n_firms_intract[tt]],
    M[tt] * expu[tt,jj] / mktdenom[tt])

firmq = @NLexpression(model,
    [jj in 1:n_firms],
    sum(mktq[tt,findall(x->x == jj, j_indexer[tt])[1]] for tt in t_indexer[jj]))

@NLobjective(model, Min, sum((firmq[jj] - q_obs[jj])^2 for jj in 1:n_firms))

optimize!(model)



******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       10

Total number of variables............................:        4
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality co

In [10]:
value.(θx)

2-element Vector{Float64}:
 -0.13496910913853621
 -0.34371765735650206

In [11]:
value.(θd)

2-element Vector{Float64}:
 -130.83756902942645
  576.9122622345478

In [12]:
value.(firmq)